In [1]:
pip install -U requests langchain langchain-aws langchain-community pydantic boto3 tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.38.46
    Uninstalling botocore-1.38.46:
      Successfully uninstalled botocore-1.38.46
  Attempting uninstall: tavily-python
    Found existing installation: tavily-python 0.7.8
    Uninstalling tavily-python-0.7.8:
      Successfully uninstalled tavily-python-0.7.8
  Attempting uninstall: boto3
    Found existing installation: boto3 1.38.46
    Uninstalling boto3-1.38.46:
      Successfully uninstalled boto3-1.38.46

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2-binary


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
FUZZY_MATCH_SYSTEM_PROMPT = """
You are a highly skilled data analyst and researcher specializing in entity resolution. Your goal is to find the *most likely* merge candidate for a source entity by weighing evidence and identifying strong similarities, even if the data is not a perfect match.

**Core Directive:**
* **NO SELF-MATCHING:** Your most important rule. You are forbidden from considering the source entity as a merge candidate. After finding potential matches, your first action in the analysis phase MUST be to remove the original source entity from that list.

**Your Comprehensive Analysis Protocol:**

1.  **Get Source Data:** First, use the `get_entity_by_id` tool to fetch the complete data for the source entity.

2.  **Iterative Search:** Build a search profile from the source entity's data. Systematically use the `find_matching_entities` tool with different terms (the entity's name, a key competition name, a key team member's name) to create a comprehensive list of potential candidates.

3.  **Consolidate & Deduplicate:** Combine all search results into a single list of unique candidates.

4.  **Apply Core Directive:** Execute the **NO SELF-MATCHING** directive by removing the source entity from your consolidated list.

5.  **Holistic Similarity Analysis:** Now, with a clean list of *other* entities, perform a holistic analysis. For each candidate, you must weigh how similar it is to the source entity based on the following factors.

**Key Factors for Comparison:**
* **Team Name:** How similar are the names? Consider partial matches, acronyms, or shared keywords. A very high similarity is a strong signal.
* **Core Attributes (`sport`, `gender`, `team type`):** These should ideally be exact matches. Note any discrepancies as significant differences.
* **Shared Context (`competitions`, `teamMembers`):** This is very important. How much overlap is there in the lists of competitions and team members? A high degree of overlap (several shared names) is a very strong indicator of a match.
* **Jersey Numbers:** Check for any overlap in `preferredJersey` numbers associated with the same team members.

**Final Answer Formatting:**
Your final answer MUST be in the required format.
- If you find a likely candidate, pick the best one and format the output like this:
Best Match ID: [GSL_ID_of_the_best_match]
Justification: [Provide a balanced argument for your decision. Your justification **must include both the similarities that support the match AND any notable differences** you found. Explain why you believe the similarities outweigh the differences.]

- If no candidate is a strong enough match after weighing the evidence, format the output like this:
Best Match ID: no match found
Justification: [Explain why no candidates met a reasonable threshold for similarity based on your analysis of the key factors.]
"""

EXACT_MATCH_SYSTEM_PROMPT = """
You are a deterministic data processing bot. Your only goal is to follow a strict set of rules to compare a source entity to potential candidates. You must operate with precision and without interpretation.

**Core Directives:**
1.  **NO SELF-MATCHING:** Your most important rule. You are forbidden from considering the source entity as a merge candidate. After finding potential matches, your first action in the analysis phase MUST be to remove the original source entity from that list.
2.  **EXACT MATCHING ONLY:** You must only recommend a merge if a candidate passes the "Exact Match Rule" defined below. There is no room for interpretation or "close" matches. If even one condition in the rule fails, the candidate is invalid.

**Your Strict Analysis Protocol:**

1.  **Get Source Data:** Use the `get_entity_by_id` tool to fetch the complete data for the source entity.

2.  **Iterative Search:** Build a search profile from the source entity's data. Systematically use the `find_matching_entities` tool with different terms (the entity's name, a key competition name, a key team member's name) to create a comprehensive list of potential candidates.

3.  **Consolidate & Deduplicate:** Combine all search results into a single list of unique candidates.

4.  **Apply Core Directives:**
    * First, execute the **NO SELF-MATCHING** directive by removing the source entity from your consolidated list.
    * Then, for every remaining candidate, apply the **Exact Match Rule** below.

**The Exact Match Rule:**
A target entity is a valid merge candidate **if and only if all** of the following conditions are true when compared to the source entity:
* `sport` must be an exact match.
* `gender` must be an exact match.
* `name` must be an exact, case-insensitive match.
* The list of `competition.name` values must have at least one exact match.
* The list of `teamMembers.individual.commonName.fullName` values must have at least one exact match.

**Final Answer Formatting:**
Your final answer MUST be in the required format.
- If one or more candidates pass the Exact Match Rule, pick the best one and format the output like this:
Best Match ID: [GSL_ID_of_the_best_match]
Justification: [Your detailed reasoning here, stating that all fields in the Exact Match Rule were met.]

- If NO candidates pass the Exact Match Rule, format the output like this:
Best Match ID: no match found
Justification: [Your reasoning for why no candidates passed the strict matching criteria.]
"""

WEIGHTED_SCORING_PROMPT = """
You are a meticulous and literal data validation bot. Your only function is to execute the following instructions precisely as written, without deviation or interpretation. You must not hallucinate or invent information. Your analysis is based solely on applying the provided scoring rubric.

**Core Directives:**
1.  **NO SELF-MATCHING:** This is your most important rule. You are forbidden from considering the source entity as a merge candidate. This check must be performed after finding candidates and before scoring.
2.  **ADHERE TO THE RUBRIC:** You must derive the final score by strictly following the "Scoring Rubric" below. Do not assign points for any criteria not listed.
3.  **NO HALLUCINATION:** Do not invent or infer information. If data for a specific field is not present in the API response, you must treat it as a non-match for that criterion.

**Your Strict Analysis Protocol:**
1.  **Get Source Data:** Fetch the complete data for the source entity using the `get_entity_by_id` tool.
2.  **Iterative Search:** Find potential candidates using the `find_matching_entities` tool.
3.  **Consolidate & Filter:** Combine all search results into a unique list and immediately execute the **NO SELF-MATCHING** directive.
4.  **Score Each Candidate:** For every remaining candidate, apply the **Scoring Rubric** below to calculate a final score.

**The Scoring Rubric (Max Score: 5):**
* **+2 points** if `sport` and `gender` are BOTH an exact match. (Award 0 points if either is different).
* **+1 point** if `name` is a very strong match (e.g., exact, near-exact, or a common acronym).
* **+1 point** if there is an overlap of **at least one** `competition.name`.
* **+1 point** if there is an overlap of **at least one** `teamMembers.individual.commonName.fullName`.

**Final Answer Formatting:**
Your final answer MUST be in the following format. Pick the candidate with the highest score.
- If the highest score is 2 or more, format it like this:
Best Match ID: [GSL_ID_of_the_best_match]
Score: [Calculated score out of 5]
Justification: [Your detailed reasoning, explaining how you calculated the score based on the rubric.]

- If no candidate scores 2 or more, format it like this:
Best Match ID: no match found
Score: [Score of the highest-scoring (but still failing) candidate, e.g., 1/5]
Justification: [Your reasoning for why no candidate met the minimum score threshold.]
"""

BINARY_SCORING_PROMPT = """
You are a deterministic data processing bot. Your only function is to execute the following instructions with absolute precision. You must not interpret, infer, or hallucinate information.

**Core Directives:**
1.  **NO SELF-MATCHING:** Your most important rule. You are forbidden from considering the source entity as a merge candidate. This check must be performed after finding candidates and before applying the match rule.
2.  **BINARY LOGIC ONLY:** Your entire analysis is based on the "Exact Match Rule" below. A candidate either passes all conditions (Score 5) or it fails (Score 0). There are no partial scores.
3.  **NO HALLUCINATION:** Only use data explicitly provided by the tools.

**Your Strict Analysis Protocol:**
1.  **Get Source Data:** Fetch the complete data for the source entity.
2.  **Iterative Search:** Find potential candidates using the `find_matching_entities` tool.
3.  **Consolidate & Filter:** Combine results into a unique list and execute the **NO SELF-MATCHING** directive.
4.  **Apply Exact Match Rule & Score:** For every remaining candidate, check if it passes **ALL** conditions in the "Exact Match Rule."
    * If it passes all conditions, assign it a **Score of 5**.
    * If it fails even one condition, assign it a **Score of 0**.

**The Exact Match Rule:**
A target entity passes if and only if **all** of the following conditions are true when compared to the source entity:
* `sport` is an exact match.
* `gender` is an exact match.
* `name` is an exact, case-insensitive match.
* The list of `competition.name` values has at least one exact match.
* The list of `teamMembers.individual.commonName.fullName` values has at least one exact match.

**Final Answer Formatting:**
Your final answer MUST be in the following format. Pick a candidate that scored a 5.
- If one or more candidates score a 5, format the output like this:
Best Match ID: [GSL_ID_of_the_best_match]
Score: 5/5
Justification: [State that the candidate passed all conditions of the Exact Match Rule.]

- If NO candidates score a 5, format the output like this:
Best Match ID: no match found
Score: 0/5
Justification: [State that no candidates passed all conditions of the strict Exact Match Rule.]
"""

In [5]:
import os
import csv
import json
import requests
import re
import base64
import psycopg2
from typing import List, Dict, Any

# LangChain Imports...
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_aws import ChatBedrock
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# --- Step 1: Securely Load Configuration ---
load_dotenv()

# API Keys
HUDL_API_KEY = os.getenv("HUDL_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# DB Credentials
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_PORT = os.getenv("DB_PORT", "5432")

# Validate that all required variables are set
if not all([HUDL_API_KEY, TAVILY_API_KEY, DB_HOST, DB_NAME, DB_USER, DB_PASS]):
    raise ValueError("One or more required environment variables are not set.")

GRAPHQL_ENDPOINT = "https://master.thorhudl.com/api/graphql/query"
HEADERS = {
    "Authorization": f"Bearer {HUDL_API_KEY}",
    "Content-Type": "application/json"
}

# ---Step 1: Tool Definitions ---
# --- Tool 1: Get Entity to be Merged ---
class MergeEntityDetailsInput(BaseModel):
    gsl_id: str = Field(description="The unique GSL ID of the source entity to be merged.")

@tool(args_schema=MergeEntityDetailsInput)
def get_entity_by_id(gsl_id: str) -> Dict[str, Any]:
    """Fetches the full details for a single entity by its unique GSL ID."""
    get_team_query = f"""
        query getTeams {{
          searchableTeams(query: [{{ field: ID, operator: EQUALS, values: ["{gsl_id}"] }}]) {{
            items {{ id name sport gender teamMembers {{ preferredJersey individual {{ id commonName {{ fullName }} }} }} competitions {{ id name }} }}
          }}
        }}
    """
    try:
        response = requests.post(GRAPHQL_ENDPOINT, headers=HEADERS, data=json.dumps({"query": get_team_query}))
        response.raise_for_status()
        data = response.json()
        if "errors" in data:
            return {"error": "GraphQL query failed.", "details": data['errors']}
        
        items = data.get("data", {}).get("searchableTeams", {}).get("items", [])
        return items[0] if items else {"error": f"No entity found with GSL ID {gsl_id}"}
        
    except requests.exceptions.RequestException as e:
        return {"error": f"API call failed: {e}"}
    except (json.JSONDecodeError, IndexError):
        return {"error": "Failed to parse API response or find entity."}


# --- Tool 2: Search for Matching Entities ---
class TargetEntityDetailsInput(BaseModel):
    search_term: str = Field(description="A name or keyword to search for matching entities.")
    
@tool(args_schema=TargetEntityDetailsInput)
def find_matching_entities(search_term: str) -> List[Dict[str, Any]]:
    """Searches for entities by a name and returns a list of potential matches."""
    get_teams_query = f"""
        query searchTeams {{
          searchableTeams(searchTerm: "{search_term}") {{
            items {{ id name sport gender teamMembers {{ preferredJersey individual {{ id commonName {{ fullName }} }} }} competitions {{ id name }} }}
          }}
        }}
    """
    try:
        response = requests.post(GRAPHQL_ENDPOINT, headers=HEADERS, data=json.dumps({"query": get_teams_query}))
        response.raise_for_status()
        data = response.json()
        if "errors" in data:
            print(f"GraphQL API returned an error: {data['errors']}")
            return []
        
        return data.get("data", {}).get("searchableTeams", {}).get("items", [])

    except requests.exceptions.RequestException as e:
        print(f"API call failed: {e}")
        return []
    except json.JSONDecodeError:
        print("Failed to parse API response.")
        return []

# --- Tool 3: The Google Search Tool ---
tavily_tool = TavilySearchResults(max_results=3)
tavily_tool.name = "tavily_search_results_json"


# --- Step 2: Create the Agent with a Modified Prompt for Parsing ---
# The list of tools
tools = [get_entity_by_id, find_matching_entities]

llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0.0}
)

# This is the core prompt - the "brain" of the agent.
# It tells the agent HOW to perform the comparison task step-by-step.

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WEIGHTED_SCORING_PROMPT),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Create the agent and executor (remains the same)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
# IDS From CSV
# --- Step 3: Main Batch Processing Logic ---

def parse_agent_output(output_text: str) -> (str, str):
    """Parses the agent's final answer to extract the best match ID and justification."""
    match_id = "no match found"
    justification = output_text # Default to the full output

    # Use regex to find the Best Match ID line
    match = re.search(r"Best Match ID:\s*(.*)", output_text)
    if match:
        match_id = match.group(1).strip()
    
    # Extract justification text that comes after the ID line
    justification_split = output_text.split("Justification:", 1)
    if len(justification_split) > 1:
        justification = justification_split[1].strip()

    return match_id, justification

def run_batch_process(input_file: str, output_file: str):
    """Reads source IDs from a CSV, processes them with the agent, and writes results to a new CSV."""
    print(f"Starting batch process from '{input_file}'...")
    
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Write header to the output file
        writer.writerow(["source_gsl_id", "best_match_gsl_id", "justification"])
        
        # Skip header row of the input file
        header = next(reader)
        print(f"Processing input file with header: {header}")

        for row in reader:
            source_id = row[0]
            print(f"\n--- Processing Source GSL ID: {source_id} ---")

            user_input = f"Please research and compare the entity with GSL ID {source_id} to find the best merge candidate. Follow your instructions precisely."
            
            try:
                response = agent_executor.invoke({"input": user_input})
                output_data = response.get("output", [])
                output_text = "" # Default to empty string

                # The response is a list like [{'type': 'text', 'text': '...'}].
                # We need to safely extract the 'text' value.
                if isinstance(output_data, list) and output_data:
                    if isinstance(output_data[0], dict) and 'text' in output_data[0]:
                        output_text = output_data[0]['text']
                elif isinstance(output_data, str): # Handle if it's already a string
                    output_text = output_data
                
                # Parse the structured output
                best_match_id, justification = parse_agent_output(output_text)

                if best_match_id != 'no match found':
                    decoded_bytes = base64.b64decode(best_match_id)
                    best_match_id = decoded_bytes.decode("utf-8")

                print(f"--- Result for {source_id}: Found match '{best_match_id}' ---")
                writer.writerow([source_id, best_match_id, justification])

            except Exception as e:
                print(f"!! An error occurred while processing {source_id}: {e} !!")
                writer.writerow([source_id, "processing error", str(e)])

    print(f"\nBatch process complete. Results saved to '{output_file}'.")


if __name__ == "__main__":
    # Define input and output file names
    input_csv_file = "/Users/suraj.salunke/Desktop/input_ids.csv"
    output_csv_file = "/Users/suraj.salunke/Desktop/output_results.csv"

    # Run the main process
    run_batch_process(input_file=input_csv_file, output_file=output_csv_file)

Starting batch process from '/Users/suraj.salunke/Desktop/input_ids.csv'...
Processing input file with header: ['source_gsl_id']

--- Processing Source GSL ID: 63f6c7e7dd4832a69b175303 ---


> Entering new AgentExecutor chain...

Invoking: `get_entity_by_id` with `{'gsl_id': '63f6c7e7dd4832a69b175303'}`
responded: [{'type': 'tool_use', 'id': 'toolu_bdrk_01SQXCVBSpsBL3BaAkpnvdYW', 'name': 'get_entity_by_id', 'input': {}, 'index': 0, 'partial_json': '{"gsl_id": "63f6c7e7dd4832a69b175303"}'}]

{'id': 'R1NMU2VhcmNoYWJsZVRlYW02M2Y2YzdlN2RkNDgzMmE2OWIxNzUzMDM=', 'name': 'HS Benilde St Margarets Red Knights', 'sport': 'ICE_HOCKEY', 'gender': 'WOMEN', 'teamMembers': [{'preferredJersey': '4', 'individual': {'id': 'R1NMSW5kaXZpZHVhbDYzZjdjMTEwZGQ3MDU5ZTMyOTAxYWYyOQ==', 'commonName': {'fullName': 'Bailey Gray'}}}, {'preferredJersey': '9', 'individual': {'id': 'R1NMSW5kaXZpZHVhbDYzZjdjMTQxZGQ3MDU5ZTMyOTA1YmE4Yw==', 'commonName': {'fullName': 'Kendall Hassler'}}}, {'preferredJersey': '44', 'individ

In [ ]:
# IDS From DB
# --- Function to Fetch IDs from PostgreSQL ---
def fetch_ids_from_postgres(limit) -> List[str]:
    """Connects to PostgreSQL and fetches the first N GSL IDs from the team table."""
    conn = None
    try:
        print("Connecting to PostgreSQL database...")
        conn = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        query = "SELECT id FROM team LIMIT %s;"
        
        with conn.cursor() as cur:
            cur.execute(query, (limit,))
            rows = [item[0] for item in cur.fetchall()]
            print(f"Successfully fetched {len(rows)} IDs from the database.")
            return rows
            
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        return [] # Return an empty list on error
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")


# --- Main Batch Processing Logic (Refactored) ---

def parse_agent_output(output_text: str) -> (str, str):
    """Parses the agent's final answer to extract the best match ID and justification."""
    # ... (function implementation remains the same) ...
    match_id = "no match found"
    justification = output_text
    match = re.search(r"Best Match ID:\s*(.*)", output_text, re.IGNORECASE)
    if match:
        match_id = match.group(1).strip()
    justification_split = re.split(r"Justification:", output_text, flags=re.IGNORECASE)
    if len(justification_split) > 1:
        justification = justification_split[1].strip()
    return match_id, justification

def run_batch_process(source_ids: List[str], output_file: str):
    """Processes a list of source IDs with the agent and writes results to a CSV."""
    print(f"Starting batch process for {len(source_ids)} IDs...")
    
    with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["source_gsl_id", "best_match_gsl_id", "justification", "decoded_match_id"])
        
        for source_id in source_ids:
            if not source_id: continue
            
            print(f"\n--- Processing Source GSL ID: {source_id} ---")
            user_input = f"Please research and compare the entity with GSL ID {source_id} to find the best merge candidate. Follow your instructions precisely."
            
            decoded_id_val = "" # Holder for the decoded ID
            try:
                response = agent_executor.invoke({"input": user_input})
                output_data = response.get("output", [])
                output_text = ""
                if isinstance(output_data, list) and output_data:
                    if isinstance(output_data[0], dict) and 'text' in output_data[0]:
                        output_text = output_data[0]['text']
                elif isinstance(output_data, str):
                    output_text = output_data
                
                best_match_id, justification = parse_agent_output(output_text)
                
                # Your Base64 decoding logic
                if best_match_id != 'no match found':
                    try:
                        decoded_bytes = base64.b64decode(best_match_id)
                        decoded_id_val = decoded_bytes.decode("utf-8")
                    except Exception as e:
                        print(f"Could not Base64 decode '{best_match_id}': {e}")
                        decoded_id_val = "decoding_error"

                print(f"--- Result for {source_id}: Found match '{best_match_id}' ---")
                writer.writerow([source_id, best_match_id, justification, decoded_id_val])

            except Exception as e:
                print(f"!! An error occurred while processing {source_id}: {e} !!")
                writer.writerow([source_id, "processing error", str(e), ""])

    print(f"\nBatch process complete. Results saved to '{output_file}'.")

if __name__ == "__main__":
    # 1. Fetch IDs from the database
    ids_to_process = fetch_ids_from_postgres(limit=100)

    # 2. Run the batch process only if IDs were successfully fetched
    if ids_to_process:
        output_csv_file = "output_results_weighted.csv"
        run_batch_process(source_ids=ids_to_process, output_file=output_csv_file)
    else:
        print("No IDs fetched from the database. Halting process.")

Connecting to PostgreSQL database...
Successfully fetched 1000 IDs from the database.
Database connection closed.
Starting batch process for 1000 IDs...

--- Processing Source GSL ID: 63ff329ab352a35c8f3b877c ---


> Entering new AgentExecutor chain...

Invoking: `get_entity_by_id` with `{'gsl_id': '63ff329ab352a35c8f3b877c'}`
responded: [{'type': 'tool_use', 'id': 'toolu_bdrk_01DXZU4JvgoerBPanypj3uwN', 'name': 'get_entity_by_id', 'input': {}, 'index': 0, 'partial_json': '{"gsl_id": "63ff329ab352a35c8f3b877c"}'}]

{'error': 'No entity found with GSL ID 63ff329ab352a35c8f3b877c'}[{'type': 'text', 'text': '\n\nThe source entity with GSL ID 63ff329ab352a35c8f3b877c could not be found. Therefore, there are no candidates to evaluate.\n\nBest Match ID: no match found\nScore: 0/5\nJustification: The source entity could not be retrieved, so there are no candidates to compare against.', 'index': 0}]

> Finished chain.
--- Result for 63ff329ab352a35c8f3b877c: Found match 'no match found' ---

--